In [10]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchviz import make_dot

from cifar10_models_qinacme import model1, model3, model2, model_torch_official
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
batch_size = 128
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='/datasets/CIFAR-10', train=True,
                                        download=False, transform=transform)
# originally: batchsize = 4, shuffle=False
# TODO: increase number of workers/batchsize = 128
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10('/datasets/CIFAR-10', train=False,
                                        download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
def train(model):
    results = []
    model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
#     optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    print_freq = 60
    for epoch in range(30):
#         running_loss = 0.
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            optimizer.zero_grad()

            outputs = model(inputs)
            
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

#             running_loss += loss.data[0]
#             if i%print_freq == print_freq-1:
#                 print('[{:<5}, {:<5}] loss: {:0.3f}'
#                       .format(epoch+1, i+1, running_loss / (print_freq*batch_size))
#                 running_loss = 0.0
        r = collect_data(model, criterion)
        print('[Epoch {:<2}]: train_acc{} train_loss{} test_acc{} test_loss{}'
              .format(epoch, r))
        results.append(r)
    print('Finished Training')

In [ ]:
def collect_data(model, criterion):
    result = []
    for loader in [trainloader,testloader]:
        correct = 0
        total = 0
        loss = 0.
        for data in loader:
            images, labels = data
            outputs = model(Variable(images.cuda()))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.cuda()).sum()
            loss += criterion(outputs, labels).data[0]
        result.append(correct / total) # accuracy
        result.append(loss / total) # loss
    return result
    

In [5]:
def test(model):
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        outputs = model(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

    print('Accuracy of the network on the 10000 test images: {:0.2f}%'
          .format(100 * correct / total))
    
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    for data in testloader:
        images, labels = data
        outputs = model(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels.cuda())
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i]
            class_total[label] += 1

    for i in range(10):
        print('Accuracy of {:<5} : {:0.2f}%'
              .format(classes[i], 100*class_correct[i]/class_total[i]))
    

In [ ]:
def visualize(model):
    
    make_dot(y, params)

In [11]:
net = model_torch_official()

In [12]:
acc, loss = train(net)

[1    , 60   ] loss: 0.063
[1    , 120  ] loss: 0.055
[1    , 180  ] loss: 0.052
[1    , 240  ] loss: 0.050
[1    , 300  ] loss: 0.048
[1    , 360  ] loss: 0.046
[2    , 60   ] loss: 0.045
[2    , 120  ] loss: 0.045
[2    , 180  ] loss: 0.044
[2    , 240  ] loss: 0.043
[2    , 300  ] loss: 0.043
[2    , 360  ] loss: 0.042
[3    , 60   ] loss: 0.041
[3    , 120  ] loss: 0.040
[3    , 180  ] loss: 0.040
[3    , 240  ] loss: 0.040
[3    , 300  ] loss: 0.040
[3    , 360  ] loss: 0.040
[4    , 60   ] loss: 0.038
[4    , 120  ] loss: 0.038
[4    , 180  ] loss: 0.038
[4    , 240  ] loss: 0.037
[4    , 300  ] loss: 0.037
[4    , 360  ] loss: 0.037
[5    , 60   ] loss: 0.036
[5    , 120  ] loss: 0.036
[5    , 180  ] loss: 0.035
[5    , 240  ] loss: 0.035
[5    , 300  ] loss: 0.036
[5    , 360  ] loss: 0.035
[6    , 60   ] loss: 0.034
[6    , 120  ] loss: 0.034
[6    , 180  ] loss: 0.034
[6    , 240  ] loss: 0.034
[6    , 300  ] loss: 0.034
[6    , 360  ] loss: 0.034
[7    , 60   ] loss: 0.033
[

In [13]:
test(net)

Accuracy of the network on the 10000 test images: 64.28%
Accuracy of plane : 67.40%
Accuracy of car   : 74.20%
Accuracy of bird  : 50.00%
Accuracy of cat   : 44.60%
Accuracy of deer  : 63.00%
Accuracy of dog   : 48.30%
Accuracy of frog  : 71.60%
Accuracy of horse : 67.70%
Accuracy of ship  : 80.60%
Accuracy of truck : 75.40%


In [14]:
net = model2()
train(net)

[1    , 60   ] loss: 0.066
[1    , 120  ] loss: 0.061
[1    , 180  ] loss: 0.059
[1    , 240  ] loss: 0.058
[1    , 300  ] loss: 0.057
[1    , 360  ] loss: 0.056
[2    , 60   ] loss: 0.055
[2    , 120  ] loss: 0.055
[2    , 180  ] loss: 0.055
[2    , 240  ] loss: 0.054
[2    , 300  ] loss: 0.054
[2    , 360  ] loss: 0.054
[3    , 60   ] loss: 0.053
[3    , 120  ] loss: 0.053
[3    , 180  ] loss: 0.053
[3    , 240  ] loss: 0.053
[3    , 300  ] loss: 0.053
[3    , 360  ] loss: 0.052
[4    , 60   ] loss: 0.052
[4    , 120  ] loss: 0.052
[4    , 180  ] loss: 0.052
[4    , 240  ] loss: 0.052
[4    , 300  ] loss: 0.052
[4    , 360  ] loss: 0.052
[5    , 60   ] loss: 0.051
[5    , 120  ] loss: 0.051
[5    , 180  ] loss: 0.051
[5    , 240  ] loss: 0.051
[5    , 300  ] loss: 0.051
[5    , 360  ] loss: 0.051
[6    , 60   ] loss: 0.050
[6    , 120  ] loss: 0.050
[6    , 180  ] loss: 0.050
[6    , 240  ] loss: 0.050
[6    , 300  ] loss: 0.050
[6    , 360  ] loss: 0.050
[7    , 60   ] loss: 0.049
[

In [15]:
test(net)

Accuracy of the network on the 10000 test images: 79.23%
Accuracy of plane : 80.80%
Accuracy of car   : 88.10%
Accuracy of bird  : 69.40%
Accuracy of cat   : 64.50%
Accuracy of deer  : 77.50%
Accuracy of dog   : 69.30%
Accuracy of frog  : 85.80%
Accuracy of horse : 82.90%
Accuracy of ship  : 88.90%
Accuracy of truck : 86.10%


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img/2+0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join(classes[labels[j]] for j in range(4)))

In [16]:
a = [1,2,3,4]

1
